In [ ]:
from collections import namedtuple, deque
from heapq import heappop, heappush
from random import choice
from tqdm.auto import tqdm
import numpy as np
from icecream import ic

In [16]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

In [17]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


def is_goal(state: np.ndarray) -> bool:
    goal = np.roll(np.arange(PUZZLE_DIM ** 2), -1).reshape((PUZZLE_DIM, PUZZLE_DIM))
    return np.array_equal(state, goal)


def fitness(state: np.ndarray) -> int:
    goal = np.roll(np.arange(PUZZLE_DIM ** 2), -1).reshape((PUZZLE_DIM, PUZZLE_DIM))
    return np.sum(state == goal)

In [18]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

In [19]:
def manhattan_distance(state: np.ndarray) -> int:
    goal_positions = {val: (i // PUZZLE_DIM, i % PUZZLE_DIM) for i, val in enumerate(np.roll(np.arange(PUZZLE_DIM ** 2), -1))}
    dist = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            goal_x, goal_y = goal_positions[state[x, y]]
            dist += abs(x - goal_x) + abs(y - goal_y)
    return dist

def a_star_search(initial_state: np.ndarray):
    fronteer = []
    heappush(fronteer, (0, initial_state.tobytes(), [], 0))
    visited = set()
    
    while fronteer:
        estimated_cost, current_state_bytes, path, g_cost = heappop(fronteer)
        
        # Convert the bytes back to an array for further processing
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))

        # Check if we reached the goal
        if is_goal(current_state):
            return path, len(path)

        # Mark the state as visited
        visited.add(current_state_bytes)

        # Explore neighbors
        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_state_bytes = new_state.tobytes()
            if new_state_bytes not in visited:
                new_g_cost = g_cost + 0.5
                estimated_cost = new_g_cost + manhattan_distance(new_state)
                heappush(fronteer, (estimated_cost, new_state_bytes, path + [action], new_g_cost))

    return None, 0  # If no solution found

print(state)
print(manhattan_distance(state))
solution_path, steps = a_star_search(state)

print(f"Number of steps to solution: {steps}")
print(state)
print("Solution path:")
for move in solution_path:
    print(move)
    state = do_action(state, move)
    print(state)


[[ 9  3  6 14]
 [13  5 11 15]
 [ 2 12  0  8]
 [10  1  4  7]]
38
Number of steps to solution: 62
[[ 9  3  6 14]
 [13  5 11 15]
 [ 2 12  0  8]
 [10  1  4  7]]
Solution path:
Action(pos1=(2, 2), pos2=(3, 2))
[[ 9  3  6 14]
 [13  5 11 15]
 [ 2 12  4  8]
 [10  1  0  7]]
Action(pos1=(3, 2), pos2=(3, 3))
[[ 9  3  6 14]
 [13  5 11 15]
 [ 2 12  4  8]
 [10  1  7  0]]
Action(pos1=(3, 3), pos2=(2, 3))
[[ 9  3  6 14]
 [13  5 11 15]
 [ 2 12  4  0]
 [10  1  7  8]]
Action(pos1=(2, 3), pos2=(2, 2))
[[ 9  3  6 14]
 [13  5 11 15]
 [ 2 12  0  4]
 [10  1  7  8]]
Action(pos1=(2, 2), pos2=(1, 2))
[[ 9  3  6 14]
 [13  5  0 15]
 [ 2 12 11  4]
 [10  1  7  8]]
Action(pos1=(1, 2), pos2=(1, 3))
[[ 9  3  6 14]
 [13  5 15  0]
 [ 2 12 11  4]
 [10  1  7  8]]
Action(pos1=(1, 3), pos2=(2, 3))
[[ 9  3  6 14]
 [13  5 15  4]
 [ 2 12 11  0]
 [10  1  7  8]]
Action(pos1=(2, 3), pos2=(2, 2))
[[ 9  3  6 14]
 [13  5 15  4]
 [ 2 12  0 11]
 [10  1  7  8]]
Action(pos1=(2, 2), pos2=(1, 2))
[[ 9  3  6 14]
 [13  5  0  4]
 [ 2 12 15 11